In [124]:
from datasets import load_dataset, DownloadMode
import torch 
import plotly.express as px
from IPython.display import Audio as DisplayAudio, display
from transformers import AutoProcessor, AutoModelForAudioClassification
from transformers import TrainingArguments, Trainer, Wav2Vec2FeatureExtractor, Wav2Vec2Model, AutoModelForAudioClassification
import numpy as np 
import pandas as pd 

import transformers
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor, pipeline
import datasets
from datasets import Audio, Dataset

In [125]:
device = torch.device(
        "cuda"
        if torch.cuda.is_available()
        else "mps" if torch.backends.mps.is_available() else "cpu"
    )

In [126]:
dataset = load_dataset("../iemocap", split="train", trust_remote_code=True, download_mode=DownloadMode.FORCE_REDOWNLOAD).with_format("torch", device=device)
num_rows = dataset.num_rows
print('number of rows: ', num_rows)
print('type: ', type(dataset))
dataset.features



Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 100%|██████████| 1819/1819 [00:00<00:00, 2958.15 examples/s]

number of rows:  1819
type:  <class 'datasets.arrow_dataset.Dataset'>


{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'text': Value(dtype='string', id=None),
 'rating1': Value(dtype='string', id=None),
 'rating2': Value(dtype='string', id=None),
 'rating3': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'valence': Value(dtype='float32', id=None),
 'activation': Value(dtype='float32', id=None),
 'dominance': Value(dtype='float32', id=None),
 'label': ClassLabel(names=['neu', 'fru', 'sad', 'sur', 'ang', 'hap', 'exc', 'fea', 'dis', 'xxx', 'oth'], id=None)}

In [127]:
audio_array_wrapper = dataset[10]["audio"]
audio_array_wrapper

{'path': '/Users/hannahmanfredi/.cache/huggingface/datasets/downloads/extracted/8ae19e95b6f4b1c54cfe7c431eaa1ac802ae3e5691b37f3a703719cf096e8838/iemocap/Session1/sentences/wav/Ses01M_impro01/Ses01M_impro01_F010.wav',
 'array': tensor([ 1.5259e-03,  6.1035e-05, -1.3428e-03,  ..., -9.7656e-04,
         -2.3804e-03, -3.7842e-03], device='mps:0'),
 'sampling_rate': tensor(16000, device='mps:0')}

In [128]:
# display(DisplayAudio(audio_array_wrapper['array'], rate=audio_array_wrapper['sampling_rate']))

In [129]:
# Convert the HuggingFace dataset to a pandas DataFrame
# df = dataset.to_pandas()
# df.head()
# df['label'] = df['label'].map(lambda label: dataset.features["label"].names[label])

# fig = px.scatter_3d(df, x='valence', y='activation', z='dominance', color='label')
# fig.update_layout(
#     autosize=False,
#     width=1000,
#     height=1000,
# )
# fig.show()

In [130]:
# Generate Histogram to visualize the distribution of the labels
# fig = px.histogram(df, x='label', title='Initial Label Distribution')
# fig.show()

In [131]:
# Balance the dataset by removing outliers

# Count the number of occurrences of each label
# label_counts = df['label'].value_counts()

# # Define a threshold for the minimum number of samples per label
# threshold = 100

# # Filter out labels with fewer than the threshold number of samples
# df = df.groupby('label').filter(lambda x: len(x) > threshold)

# fig = px.histogram(df, x='label', title='Balanced Label Distribution')
# fig.show()



In [132]:
batch_size = 32
sampling_rate = 16000
sample = dataset[192]

# processor = AutoProcessor.from_pretrained("audeering/wav2vec2-large-robust-12-ft-emotion-msp-dim")
model = AutoModelForAudioClassification.from_pretrained("hughlan1214/Speech_Emotion_Recognition_wav2vec2-large-xlsr-53_240304_SER_fine-tuned2.0")

feature_extractor = AutoFeatureExtractor.from_pretrained("hughlan1214/Speech_Emotion_Recognition_wav2vec2-large-xlsr-53_240304_SER_fine-tuned2.0")
dataset = dataset.remove_columns(['text', 'rating1', 'rating2', 'rating3', 'gender', 'valence', 'activation', 'dominance'])
inputs = feature_extractor(sample["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")

# def prepare_dataset(examples):
#    # for each ex. in my batch, go into my audio obj and get my raw audio data
#    audio_arrays = [x["array"].cpu().numpy() for x in examples["audio"]]
#    inputs = processor(
#         audio_arrays,
#         sampling_rate=sampling_rate,
#         truncation=True,
#         padding="longest",
#         max_length=3 * sampling_rate,
#     )
#    return inputs

# #apply our processor to our dataset
# dataset = dataset.map(prepare_dataset, remove_columns='audio', batched=True, batch_size=batch_size)
# inputs = dataset
inputs
[sample["label"], sample["rating1"], sample["rating2"], sample["rating3"]]


[tensor(6, device='mps:0'), 'Excited', 'Surprise', 'Excited']

## Inference

In [133]:
with torch.no_grad():
    logits = model(**inputs).logits

In [134]:
predicted_class_ids = torch.argmax(logits).item()
predicted_label = model.config.id2label[predicted_class_ids]
predicted_label


'happy'